In [1]:
import pandas as pd
import sys
import os
import glob

In [2]:
os.chdir("/ufrc/soltis/shan158538/TragFL_NewAnalysis/OutPut/DE_analysis/DESeq2_Homeolog-specific-expression_Tms/")

In [14]:
## Import 5,118 orthologs in file bayes_flag_sig_Tms_for_UR.csv; these orthologs showed non-biased mapping and were analyzed for homeolog-specific expression analysis (not all 5,400 orthologs showing unbaised mapping have homeolog-specific expression profiles)
Homeolog_expression_profile = pd.read_table("bayes_flag_sig_Tms_for_UR.csv", sep = ",", index_col = False)

In [15]:
Homeolog_expression_profile.head()

,commonID,q4_mean_theta,q4_q025,q4_q975,q5_mean_theta,q5_q025,q5_q975,q6_mean_theta,q6_q025,q6_q975,flag_q4_sig,flag_q5_sig,flag_q6_sig,flag_sig_Tms_tdu_tpr
0,Tpr_TRINITY_DN11281_c1_g9|Tdu_TRINITY_DN15992_...,0.712,0.406,0.896,0.669,0.304,0.893,0.645,0.271,0.879,0,0,0,0
1,Tpr_TRINITY_DN11283_c3_g1|Tdu_TRINITY_DN25596_...,0.608,0.418,0.786,0.529,0.333,0.708,0.446,0.259,0.637,0,0,0,0
2,Tpr_TRINITY_DN11283_c4_g7|Tdu_TRINITY_DN21302_...,0.297,0.122,0.491,0.229,0.092,0.406,0.168,0.060,0.317,1,1,1,1
3,Tpr_TRINITY_DN11284_c2_g19|Tdu_TRINITY_DN20652...,0.527,0.284,0.758,0.447,0.202,0.699,0.384,0.169,0.620,0,0,0,0
4,Tpr_TRINITY_DN11285_c1_g4|Tdu_TRINITY_DN17239_...,0.187,0.093,0.299,0.136,0.065,0.224,0.096,0.048,0.169,1,1,1,1


## Processing noneDE loci between Tdu and Tpr

In [17]:
## Import noneDE loci between Tdu and Tpr, based on DESeq2 results
DESeq2_noneDE_Tdu_Tpr = pd.read_table("DESeq2_none-DE_Tdu_Tpr.txt")

In [18]:
DESeq2_noneDE_Tdu_Tpr.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Tpr_TRINITY_DN11257_c2_g1|Tdu_TRINITY_DN16696_c0_g1,110.539342,-0.234395,0.191056,-1.227029,0.219812,0.445769
Tpr_TRINITY_DN10853_c1_g4|Tdu_TRINITY_DN21860_c3_g3,909.392019,-0.121086,0.141581,-0.855253,0.392411,0.626296
Tpr_TRINITY_DN12798_c2_g2|Tdu_TRINITY_DN24179_c2_g3,51.645391,-0.665939,0.258235,-2.579510,0.009894,0.052118
Tpr_TRINITY_DN21120_c0_g1|Tdu_TRINITY_DN19044_c2_g2,1.634749,0.949137,0.614108,1.765781,0.077433,0.228848
Tpr_TRINITY_DN15495_c0_g2|Tdu_TRINITY_DN17403_c2_g1,54.042560,0.055653,0.327908,0.169725,0.865227,0.941065


In [24]:
## Creat a list which contains all indexes from dataframe DESeq2_noneDE_Tdu_Tpr
List_nonDE = list(DESeq2_noneDE_Tdu_Tpr.index)

In [26]:
List_nonDE[:5]

['Tpr_TRINITY_DN11257_c2_g1|Tdu_TRINITY_DN16696_c0_g1',
 'Tpr_TRINITY_DN10853_c1_g4|Tdu_TRINITY_DN21860_c3_g3',
 'Tpr_TRINITY_DN12798_c2_g2|Tdu_TRINITY_DN24179_c2_g3',
 'Tpr_TRINITY_DN21120_c0_g1|Tdu_TRINITY_DN19044_c2_g2',
 'Tpr_TRINITY_DN15495_c0_g2|Tdu_TRINITY_DN17403_c2_g1']

In [27]:
## Identify those loci with homeolog expression profiles and showed non-differential exp between Tdu and Tpr
Tms_homeolog_exp_noneDE = Homeolog_expression_profile[Homeolog_expression_profile.commonID.isin(List_nonDE)]

In [28]:
## Showed the number of noneDE loci with homeolog expression profiles
Tms_homeolog_exp_noneDE.shape

(3925, 14)

In [29]:
Tms_homeolog_exp_noneDE.head()

,commonID,q4_mean_theta,q4_q025,q4_q975,q5_mean_theta,q5_q025,q5_q975,q6_mean_theta,q6_q025,q6_q975,flag_q4_sig,flag_q5_sig,flag_q6_sig,flag_sig_Tms_tdu_tpr
1,Tpr_TRINITY_DN11283_c3_g1|Tdu_TRINITY_DN25596_...,0.608,0.418,0.786,0.529,0.333,0.708,0.446,0.259,0.637,0,0,0,0
2,Tpr_TRINITY_DN11283_c4_g7|Tdu_TRINITY_DN21302_...,0.297,0.122,0.491,0.229,0.092,0.406,0.168,0.060,0.317,1,1,1,1
3,Tpr_TRINITY_DN11284_c2_g19|Tdu_TRINITY_DN20652...,0.527,0.284,0.758,0.447,0.202,0.699,0.384,0.169,0.620,0,0,0,0
4,Tpr_TRINITY_DN11285_c1_g4|Tdu_TRINITY_DN17239_...,0.187,0.093,0.299,0.136,0.065,0.224,0.096,0.048,0.169,1,1,1,1
5,Tpr_TRINITY_DN11294_c1_g1|Tdu_TRINITY_DN19017_...,0.144,0.000,0.535,0.117,0.000,0.504,0.094,0.000,0.413,0,0,1,0


In [43]:
## Identify noneDE loci with homeolog expression profiles that do NOT show bias expression to any subgenome
Tms_homeolog_exp_noneDE_nonebias = Tms_homeolog_exp_noneDE[Tms_homeolog_exp_noneDE["flag_sig_Tms_tdu_tpr"] == 0]
Tms_homeolog_exp_noneDE_nonebias["commonID"].to_csv("Tms_noneDE_unbias.txt", index = False)

In [44]:
## Identify noneDE loci with homeolog expression profiles that showed biased expression to Tdu subgenome
## flg=1 and q5_mean < 0.5 indicate biased expression to Tdu subgenome
Tms_homeolog_exp_noneDE_TduBias = Tms_homeolog_exp_noneDE[(Tms_homeolog_exp_noneDE["flag_sig_Tms_tdu_tpr"] == 1) & (Tms_homeolog_exp_noneDE["q5_mean_theta"] < 0.5)]
Tms_homeolog_exp_noneDE_TduBias["commonID"].to_csv("Tms_noneDE_biasTdu.txt", index = False)

In [45]:
## Identify noneDE loci with homeolog expression profiles that showed biased expression to Tpr subgenome
## flg=1 and q5_mean > 0.5 indicate biased expression to Tpr subgenome
Tms_homeolog_exp_noneDE_TprBias = Tms_homeolog_exp_noneDE[(Tms_homeolog_exp_noneDE["flag_sig_Tms_tdu_tpr"] == 1) & (Tms_homeolog_exp_noneDE["q5_mean_theta"] > 0.5)]
Tms_homeolog_exp_noneDE_TprBias["commonID"].to_csv("Tms_noneDE_biasTpr.txt", index = False)

## Processing DE loci with higher exp in Tdu

In [46]:
## Import DE loci with higher expression in Tdu, based on DESeq2 results
DESeq2_DE_Tdu_higher = pd.read_table("DESeq2_DE_higher_Tdu.txt")

In [49]:
## Creat a list which contains all indexes from dataframe DESeq2_DE_Tdu_higher
List_TduHigher = list(DESeq2_DE_Tdu_higher.index)

In [51]:
## Identify those loci with homeolog expression profiles and showed higher exp in Tdu
Tms_homeolog_exp_TduHigher = Homeolog_expression_profile[Homeolog_expression_profile.commonID.isin(List_TduHigher)]

In [52]:
## Showed the number of Tdu higher exp loci with homeolog expression profiles
Tms_homeolog_exp_TduHigher.shape

(440, 14)

In [53]:
## Identify Tdu higher loci with homeolog expression profiles that do NOT show bias expression to any subgenome
Tms_homeolog_exp_TduHigher_nonebias = Tms_homeolog_exp_TduHigher[Tms_homeolog_exp_TduHigher["flag_sig_Tms_tdu_tpr"] == 0]
Tms_homeolog_exp_TduHigher_nonebias["commonID"].to_csv("Tms_HigherTdu_unbias.txt", index = False)

## Identify Tdu higher loci with homeolog expression profiles that showed biased expression to Tdu subgenome
## flg=1 and q5_mean < 0.5 indicate biased expression to Tdu subgenome
Tms_homeolog_exp_TduHigher_TduBias = Tms_homeolog_exp_TduHigher[(Tms_homeolog_exp_TduHigher["flag_sig_Tms_tdu_tpr"] == 1) & (Tms_homeolog_exp_TduHigher["q5_mean_theta"] < 0.5)]
Tms_homeolog_exp_TduHigher_TduBias["commonID"].to_csv("Tms_HigherTdu_biasTdu.txt", index = False)

## Identify Tdu higher loci with homeolog expression profiles that showed biased expression to Tpr subgenome
## flg=1 and q5_mean > 0.5 indicate biased expression to Tpr subgenome
Tms_homeolog_exp_TduHigher_TprBias = Tms_homeolog_exp_TduHigher[(Tms_homeolog_exp_TduHigher["flag_sig_Tms_tdu_tpr"] == 1) & (Tms_homeolog_exp_TduHigher["q5_mean_theta"] > 0.5)]
Tms_homeolog_exp_TduHigher_TprBias["commonID"].to_csv("Tms_HigherTdu_biasTpr.txt", index = False)

## Processing DE loci with higher exp in Tpr

In [54]:
## Import DE loci with higher expression in Tpr, based on DESeq2 results
DESeq2_DE_Tpr_higher = pd.read_table("DESeq2_DE_higher_Tpr.txt")
## Creat a list which contains all indexes from dataframe DESeq2_DE_Tpr_higher
List_TprHigher = list(DESeq2_DE_Tpr_higher.index)

In [56]:
## Identify those loci with homeolog expression profiles and showed higher exp in Tpr
Tms_homeolog_exp_TprHigher = Homeolog_expression_profile[Homeolog_expression_profile.commonID.isin(List_TprHigher)]

## Showed the number of Tpr higher exp loci with homeolog expression profiles
Tms_homeolog_exp_TprHigher.shape

(519, 14)

In [57]:
## Identify Tpr higher loci with homeolog expression profiles that do NOT show bias expression to any subgenome
Tms_homeolog_exp_TprHigher_nonebias = Tms_homeolog_exp_TprHigher[Tms_homeolog_exp_TprHigher["flag_sig_Tms_tdu_tpr"] == 0]
Tms_homeolog_exp_TprHigher_nonebias["commonID"].to_csv("Tms_HigherTpr_unbias.txt", index = False)

## Identify Tpr higher loci with homeolog expression profiles that showed biased expression to Tdu subgenome
## flg=1 and q5_mean < 0.5 indicate biased expression to Tdu subgenome
Tms_homeolog_exp_TprHigher_TduBias = Tms_homeolog_exp_TprHigher[(Tms_homeolog_exp_TprHigher["flag_sig_Tms_tdu_tpr"] == 1) & (Tms_homeolog_exp_TprHigher["q5_mean_theta"] < 0.5)]
Tms_homeolog_exp_TprHigher_TduBias["commonID"].to_csv("Tms_HigherTpr_biasTdu.txt", index = False)

## Identify Tpr higher loci with homeolog expression profiles that showed biased expression to Tpr subgenome
## flg=1 and q5_mean > 0.5 indicate biased expression to Tpr subgenome
Tms_homeolog_exp_TprHigher_TprBias = Tms_homeolog_exp_TprHigher[(Tms_homeolog_exp_TprHigher["flag_sig_Tms_tdu_tpr"] == 1) & (Tms_homeolog_exp_TprHigher["q5_mean_theta"] > 0.5)]
Tms_homeolog_exp_TprHigher_TprBias["commonID"].to_csv("Tms_HigherTpr_biasTpr.txt", index = False)